In [1]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
from serv.poem_search import search_poems

app = Flask(__name__)
CORS(app)
#CORS(app, resources={r"/process_data": {"origins": "*"}})

@app.route('/poem_request', methods=['POST'])
def process_data():
    request_data = request.json
    print(request_data)

    result_data = search_poems(request_data['request_text'][:1000], request_data['top_n'], request_data['search_priority'])
    
    return jsonify(result_data)
    

@app.route("/")
def index():
    return "<h1>Hello!</h1>"

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)
    # from waitress import serve
    # serve(app, host="0.0.0.0", port=8080)


e:\PROGRAMS\conda\envs\torch_lm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.1.110:8080
Press CTRL+C to quit
127.0.0.1 - - [11/Feb/2024 17:14:25] "OPTIONS /poem_request HTTP/1.1" 200 -


{'request_text': 'поэт и поэзия', 'top_n': 33, 'search_priority': 0.9500000000000001}


127.0.0.1 - - [11/Feb/2024 17:14:40] "POST /poem_request HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2024 17:16:00] "OPTIONS /poem_request HTTP/1.1" 200 -


...

127.0.0.1 - - [11/Feb/2024 20:34:06] "POST /poem_request HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2024 20:50:10] "OPTIONS /poem_request HTTP/1.1" 200 -


{'request_text': 'одиночество, Марина Цветаева', 'top_n': 33, 'search_priority': 0.7700000000000001}


127.0.0.1 - - [11/Feb/2024 20:50:14] "POST /poem_request HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2024 20:50:29] "OPTIONS /poem_request HTTP/1.1" 200 -


{'request_text': 'одиночество, Марина Цветаева', 'top_n': 33, 'search_priority': 0.347}


127.0.0.1 - - [11/Feb/2024 20:50:33] "POST /poem_request HTTP/1.1" 200 -
